In [2]:
import pandas as pd
import os
import urllib.parse
import re

# === CONFIG ===
input_csv = "Marksheet_Calling_List.csv"   # CSV with Parent Name and Mobile Number
output_folder = "."      # Folder to save the HTML file
output_html = "CallingListParents.html"
mark_sheet_link = "https://forms.gle/YgKUfYDDkNWTYJHUA"  # Marksheet submission link

# === CREATE OUTPUT FOLDER IF NOT EXISTS ===
os.makedirs(output_folder, exist_ok=True)

# === READ CSV ===
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# Try to find appropriate column names for parent name and mobile number
name_col = None
mobile_col = None
for c in df.columns:
    low = c.lower().replace('_',' ').strip()
    if low in ["parent name", "parentname", "name", "guardian name", "parent"] and name_col is None:
        name_col = c
    if low in ["mobile number", "mobile", "phone", "mobilenumber", "phone number"] and mobile_col is None:
        mobile_col = c
# Fallbacks
if name_col is None and len(df.columns) >= 1:
    name_col = df.columns[0]
if mobile_col is None and len(df.columns) >= 2:
    mobile_col = df.columns[1]

# === BUILD HTML ===
html = """
<html>
<head>
<meta charset="UTF-8">
<title>Calling List - Parents (Marksheet)</title>
<style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    h2 { text-align: center; }
    table { width: 100%; border-collapse: collapse; margin-top: 20px; }
    th, td { border: 1px solid #ccc; padding: 8px; text-align: left; }
    th { background-color: #f2f2f2; }
    a { color: #007bff; text-decoration: none; }
    a:hover { text-decoration: underline; }
</style>
</head>
<body>
    <h2>Calling List - Parents (Marksheet Submission)</h2>
    <table>
        <tr>
            <th>#</th>
            <th>Parent Name</th>
            <th>Mobile Number</th>
            <th>Actions</th>
        </tr>
"""

# === POPULATE ROWS ===
i = 1
for _, row in df.iterrows():
    parent = str(row.get(name_col, '')).strip() if name_col else ''
    mobile = str(row.get(mobile_col, '')).strip() if mobile_col else ''
    # Clean mobile (keep digits only)
    mobile_digits = re.sub(r'\D', '', mobile)
    if not mobile_digits:
        mobile_digits = ''

    # Message in Hindi (as requested) — include the marksheet link
    whatsapp_message = (
        f"🌸 जय महेश! नमस्कार {parent} जी 🙏\n\n"
        "आपने पिछले मालाड उत्सव में अपने बच्चों का **मार्कशीट** सबमिट किया था। "
        "इस वर्ष का महेश्वरी प्रगति मंडल, मालाड क्षेत्रीय समितियों द्वारा आयोजित मालाड उत्सव 2025 \n"
        "🎉 दिनांक 9 नवम्बर को आयोजित किया जा रहा है।\n\n"
        "आपसे विनम्र अनुरोध है कि कृपया अपने बच्चों का मार्कशीट निम्नलिखित लिंक के माध्यम से सबमिट करें —\n"
        f"{mark_sheet_link}\n\n"
        "आपका सहयोग हमें सम्मान एवं पुरस्कार वितरण की योजना को सुचारु रूप से बनाने में मदद करेगा। 🙏\n\n"
        "– मालाड उत्सव समिति 2025\n\n"
    )

    # URL-encode for use in links
    encoded_message = urllib.parse.quote_plus(whatsapp_message)

    tel_link = f"<a onclick=\"this.style.color='green'\" href='tel:+91{mobile_digits}'>+91 {mobile_digits}</a>" if mobile_digits else ''
    sms_link = f"<a href=\"sms:+91{mobile_digits}?body={encoded_message}\" target=\"_blank\" onclick=\"this.style.color='green'\">SMS</a>" if mobile_digits else ''
    whatsapp_link = f"<a href=\"https://wa.me/91{mobile_digits}?text={encoded_message}\" target=\"_blank\" onclick=\"this.style.color='green'\">WhatsApp</a>" if mobile_digits else ''

    # Combine actions without stray newline
    actions = ' &nbsp;<br/>|&nbsp; '.join([a for a in [tel_link, sms_link, whatsapp_link] if a])
    #actions = ' &nbsp;|&nbsp; '.join([a for a in [sms_link] if a])

    html += f"""
        <tr>
            <td>{i}</td>
            <td>{parent}</td>
            <td>{mobile_digits}</td>
            <td>{actions}</td>
        </tr>
    """
    i += 1

# Close table and save
html += """
    </table>
</body>
</html>
"""

output_path = os.path.join(output_folder, output_html)
with open(output_path, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Parent calling HTML created: {output_path}")

✅ Parent calling HTML created: .\CallingListParents.html


In [4]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 6.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.0 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.0 MB 4.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/11.0 MB 4.3 MB/s eta 0:00:02
   ---------------- ----------------------- 4.5/11.0 MB 4.2 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/11.0 MB 4.1 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.0 MB 4.1 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.0 MB 4.1 MB/s eta 0:00:02
   --------------------------- ------------ 7.6/11.0 MB 4.1 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.0 MB 4.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 4.0 MB/s eta 0:00:01
   --


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\ratis\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
